# Test SparkAgent Not Always Use A Tool
By default, the ZeroShotAgent forces AI to use a tool each time which is not always the case. When we ask for generic assistance, AI should be allowed to use its power to generate answer directly, such as top 2 options for a question.

In this notebook, we experiment the newly added FreeStyleTool which allows AI to generate its own answer without using any of the provided tools.

In [1]:
from pathlib import Path
db_address = Path("./spark-warehouse/")
if (db_address.exists()):
    import shutil
    shutil.rmtree(db_address)

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
schema = "langchain_example"
spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema}")
spark.sql(f"USE {schema}")
csv_file_path = "../titanic.csv"
table = "titanic"
spark.read.csv(csv_file_path, header=True, inferSchema=True).write.saveAsTable(table, mode="overwrite")
spark.table(table).show()

23/05/13 21:35:45 WARN Utils: Your hostname, Mikes-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.68.115 instead (on interface en1)
23/05/13 21:35:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/13 21:35:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/13 21:35:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [2]:
from langchain.agents import create_spark_analytics_agent_verified

agent_executor = create_spark_analytics_agent_verified(
    schema=schema,
    allow_freestyle=True,
)




RETURN FREE STYLE





In [4]:
agent_executor.run("give me two different ways to query the most expensive class")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: 
tool args 

tool kwargs

Observation: titanic
Thought:I should check the schema of the titanic table to find the relevant columns for the most expensive class.
Action: schema_sql_db
Action Input: titanic
tool args titanic

tool kwargs

Observation: CREATE TABLE langchain_example.titanic (
  PassengerId INT,
  Survived INT,
  Pclass INT,
  Name STRING,
  Sex STRING,
  Age DOUBLE,
  SibSp INT,
  Parch INT,
  Ticket STRING,
  Fare DOUBLE,
  Cabin STRING,
  Embarked STRING)
;

/*
3 rows from titanic table:
PassengerId	Survived	Pclass	Name	Sex	Age	SibSp	Parch	Ticket	Fare	Cabin	Embarked
1	0	3	Braund, Mr. Owen Harris	male	22.0	1	0	A/5 21171	7.25	None	S
2	1	1	Cumings, Mrs. John Bradley (Florence Briggs Thayer)	female	38.0	1	0	PC 17599	71.2833	C85	C
3	1	3	Heikkinen, Miss. Laina	female	26.0	0	0	STON/O2. 3101282	7.925	None	S
*/
Thought:I can use the Pclass and Fare columns to find the most expensive class. I will pr

'There are two different ways to query the most expensive class: \n1. Using the MAX function on the Fare column: SELECT Pclass, MAX(Fare) as Max_Fare FROM titanic GROUP BY Pclass ORDER BY Max_Fare DESC LIMIT 1. The result is Class 1 with a maximum fare of 512.3292.\n2. Using the MIN function on the Pclass column: SELECT Pclass, MIN(Pclass) as Min_Pclass FROM titanic GROUP BY Pclass ORDER BY Min_Pclass ASC LIMIT 1. The result is Class 1 with a minimum Pclass value of 1.'